In [103]:
import pandas as pd
import datetime
import ccxt

In [104]:
print(ccxt.exchanges)

['ace', 'alpaca', 'ascendex', 'bequant', 'bigone', 'binance', 'binancecoinm', 'binanceus', 'binanceusdm', 'bingx', 'bit2c', 'bitbank', 'bitbay', 'bitbns', 'bitcoincom', 'bitfinex', 'bitfinex2', 'bitflyer', 'bitforex', 'bitget', 'bithumb', 'bitmart', 'bitmex', 'bitopro', 'bitpanda', 'bitrue', 'bitso', 'bitstamp', 'bitstamp1', 'bittrex', 'bitvavo', 'bl3p', 'blockchaincom', 'btcalpha', 'btcbox', 'btcmarkets', 'btctradeua', 'btcturk', 'bybit', 'cex', 'coinbase', 'coinbaseprime', 'coinbasepro', 'coincheck', 'coinex', 'coinfalcon', 'coinmate', 'coinone', 'coinsph', 'coinspot', 'cryptocom', 'currencycom', 'delta', 'deribit', 'digifinex', 'exmo', 'fmfwio', 'gate', 'gateio', 'gemini', 'hitbtc', 'hitbtc3', 'hollaex', 'huobi', 'huobijp', 'huobipro', 'idex', 'independentreserve', 'indodax', 'kraken', 'krakenfutures', 'kucoin', 'kucoinfutures', 'kuna', 'latoken', 'lbank', 'lbank2', 'luno', 'lykke', 'mercado', 'mexc', 'mexc3', 'ndax', 'novadax', 'oceanex', 'okcoin', 'okex', 'okex5', 'okx', 'paymium'

In [105]:
exchange = ccxt.binance()

In [106]:
# Retrieve trading pairs
markets = exchange.load_markets()

# Create a DataFrame from the list of trading pairs
df_currencies = pd.DataFrame(list(markets.values()))

In [107]:
# If you want to modify the original DataFrame in-place to remove rows not equal to 'EUR', you can do:
df_currencies = df_currencies[df_currencies['quote'] == 'USD']
df_currencies = df_currencies[df_currencies['type'] == 'swap']

# If you want to reset the index after dropping rows, you can use the reset_index method:
df_currencies = df_currencies.reset_index(drop=True)
df_currencies.head()
df_currencies.to_excel('Test.xlsx')

In [108]:
df_currencies['id'].describe()

count              41
unique             41
top       BTCUSD_PERP
freq                1
Name: id, dtype: object

In [109]:
df_currencies.columns

Index(['id', 'symbol', 'base', 'quote', 'baseId', 'quoteId', 'active', 'type',
       'linear', 'inverse', 'spot', 'swap', 'future', 'option', 'margin',
       'contract', 'contractSize', 'expiry', 'expiryDatetime', 'optionType',
       'strike', 'settle', 'settleId', 'precision', 'limits', 'info',
       'percentage', 'feeSide', 'tierBased', 'taker', 'maker', 'lowercaseId',
       'created'],
      dtype='object')

Retrieval of Data in a set period

In [149]:
# Define a list of trading pairs
symbols = ['XRP/USD', 'BTC/USDT', 'ETH/USDT']  # Replace with your desired list of trading pairs

# Specify the timeframe
timeframe = '5m'  # Replace with the desired timeframe

start_date = exchange.parse8601('2023-11-05T00:00:00Z')  # Start date in ISO8601 format
end_date = exchange.parse8601('2023-11-06T00:00:00Z')  # End date in ISO8601 format

limit = 1000  # Number of data points to fetch per request

# Create an empty DataFrame to store the data
df = pd.DataFrame()

while start_date < end_date:
    data_available = False  # Flag to check if data is available for at least one symbol
    max_timestamp = None  # To keep track of the maximum timestamp
    
    for symbol in symbols:
        # Calculate the end timestamp for this data request
        end_timestamp = min(start_date + limit * 300 * 1000, end_date)
        
        # Fetch the OHLCV data for each trading pair within the specified date range
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, start_date, limit, params={'endTime': end_timestamp})
        
        if ohlcv:
            data_available = True  # Data is available for at least one symbol
            
            # Extract the 'Open' and 'Volume' values and store them in a DataFrame
            open_values = [candle[1] for candle in ohlcv]
            volume_values = [candle[5] for candle in ohlcv]
            
            # Create a DataFrame for the current pair
            df_pair = pd.DataFrame({
                'Timestamp': [candle[0] for candle in ohlcv],
                f'{symbol}_Open': open_values,  # Rename the open column
                f'{symbol}_Volume': volume_values  # Create a new volume column
            })
            
            if df.empty:
                df = df_pair
            else:
                # Merge the DataFrames on the 'Timestamp' column with specified suffixes
                df = df.merge(df_pair, on='Timestamp', how='outer', suffixes=('', f'_{symbol}'))
            
            # Update the maximum timestamp for all pairs
            max_symbol_timestamp = ohlcv[-1][0]
            if max_timestamp is None or max_symbol_timestamp > max_timestamp:
                max_timestamp = max_symbol_timestamp
        
    # Update the start time for the next request
    start_date = max_timestamp + 1
    
    # Check if there are more data points to fetch, if not, exit the loop
    if not data_available or start_date >= end_date:
        break

# Sort the DataFrame by 'Timestamp'
df.sort_values(by='Timestamp', inplace=True)

# Convert the 'Timestamp' column to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')

# Reset the index
df.reset_index(drop=True, inplace=True)

# Export the DataFrame to an Excel file
df.to_excel('Pair Data.xlsx')

df

,Timestamp,XRP/USD_Open,XRP/USD_Volume,BTC/USDT_Open,BTC/USDT_Volume,ETH/USDT_Open,ETH/USDT_Volume
0,2023-11-05 00:00:00,0.6153,2.165472e+06,35062.06,85.63070,1855.54,1358.9693
1,2023-11-05 00:05:00,0.6172,3.974041e+05,35062.67,227.90551,1855.86,1184.8158
2,2023-11-05 00:10:00,0.6156,3.325032e+05,34984.61,149.21091,1852.52,2083.0947
3,2023-11-05 00:15:00,0.6152,1.525337e+05,34954.68,86.68269,1849.84,1845.0800
4,2023-11-05 00:20:00,0.6156,1.379040e+05,34987.66,52.93652,1851.35,715.5087
...,...,...,...,...,...,...,...
284,2023-11-05 23:40:00,0.6587,4.928876e+05,35090.49,61.16640,1894.89,361.0531
285,2023-11-05 23:45:00,0.6573,3.249769e+05,35054.12,56.16209,1893.15,662.9720
286,2023-11-05 23:50:00,0.6581,1.218070e+06,35034.63,45.59568,1893.23,375.1390
287,2023-11-05 23:55:00,0.6605,1.301163e+06,35026.71,44.58570,1892.94,937.3415


The choice of a time frame for a momentum trading bot depends on your trading strategy, risk tolerance, and the assets you are trading. Different time frames offer different trade-offs, and there is no one-size-fits-all answer. Here are some considerations for selecting a time frame:

1-Minute (or Less) Time Frame:
- Very short time frames like 1 minute are often used by high-frequency traders (HFT) and algorithmic trading systems that execute many trades in a short time. They require low latency and advanced infrastructure.
- High-frequency trading is associated with increased transaction costs, such as trading fees and potential slippage, and may not be suitable for all traders.
- The price action can be more erratic, making it challenging to identify meaningful trends, and there's a risk of "noise" in the data.
  
Shorter Time Frames (5-Minute, 15-Minute, 30-Minute):
- These time frames are often used by day traders and short-term momentum traders.
- They allow for more trades throughout the day but still require careful risk management due to the rapid price changes.

Longer Time Frames (1-Hour, 4-Hour, Daily):
- Longer time frames are generally used by swing traders and medium-term traders.
- Trends and patterns on these time frames tend to be more reliable, with less "noise" compared to shorter time frames.
- Trades occur less frequently, and holding positions for longer periods may require more patience.
- When choosing a time frame for a momentum trading bot, consider the following factors:

Trading Style: Your trading style and objectives matter. If you want to capitalize on very short-term price movements, you might choose a shorter time frame. For longer-term trades, consider longer time frames.

Asset Volatility: The volatility of the asset you're trading is crucial. Highly volatile assets might require shorter time frames to capture momentum, while less volatile assets can be traded on longer time frames.

Risk Management: Shorter time frames often require tighter stop-loss orders and more frequent monitoring, so you need a robust risk management strategy.

Technical Analysis: The time frame you choose should align with your technical analysis. Different time frames may show different patterns and signals.

Infrastructure: High-frequency trading often demands low-latency infrastructure, including high-speed internet connections and powerful servers.

Backtesting: Use backtesting to evaluate the performance of your trading strategy on historical data across various time frames. This can help you determine which time frame works best for your specific strategy.